In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob

In [5]:
#tea-investor-ex-communist.trycloudflare.com

In [6]:
#if you upload a unzip file to JupyterLab
#import zipfile as zf
#files = zf.ZipFile("/home/jovyan/data.zip", 'r')
#files.extractall('/home/jovyan/')
#files.close()

In [7]:
df = pd.DataFrame()
initial = "/content/drive/MyDrive/Colab Notebooks/" 
#initial = ""
files = glob.glob(initial + "data/*")
course_data = glob.glob(initial + "data/uc.berkeley.student.course.data.*.txt")
majors = glob.glob(initial + "data/uc.berkeley.student.majors.*.txt")
pathways = glob.glob(initial + "data/uc.berkeley.stem.*.txt")
passes = glob.glob(initial + "data/jacobs.*")


In [8]:
# cell to check files added to variables
passes

['/content/drive/MyDrive/Colab Notebooks/data/jacobs.participants.txt']

In [9]:
stem_path = pd.read_table(pathways[0])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (33,43,44,50,51,53,55,56,57,58,59,60,61,62,63,64,65,66,67,72,73,74,75,76,77,78,79,80,81,82,83,89,90,92,93,94,95,96,97,98,99,100,101,102,103) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [10]:
pathways

['/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.stem.pathways.2007.2017.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.stem.pathways.2007.2018.txt']

In [11]:
stem_path

,ppsk,count,year,term,entry,residency,gender,ethnic.l1,ethnic.l2,ethnic.l3,probation.year.1,GradYr,RetainY1,RetainY2,RetainY3,RetainY4,RetainY5,RetainY6,hs.gpa.unweighted,hs.gpa.weighted,sat.total,sat.math,sat.verbal,sat.writing,act.composite,act.english,act.writing,act.math,act.reading,act.science,first.generation.college.1,first.generation.college.2,hs.api,ceeb,income.parent,income.student,major.intended.college,major.intended.division,major.intended.department,esl,...,final.major.college.2,final.major.division.2,final.major.department.2,final.major.major.2,first.major.college.3,first.major.division.3,first.major.department.3,first.major.major.3,final.major.college.3,final.major.division.3,final.major.department.3,final.major.major.3,first.major.college.4,first.major.division.4,first.major.department.4,first.major.major.4,final.major.college.4,final.major.division.4,final.major.department.4,final.major.major.4,year.degree,term.degree,academic.year.degree,gpa.cum.degree,degree.college.1,degree.division.1,degree.department.1,degree.major.1,degree.college.2,degree.division.2,degree.department.2,degree.major.2,degree.college.3,degree.division.3,degree.department.3,degree.major.3,degree.college.4,degree.division.4,degree.department.4,degree.major.4
0,1023906,1,2011,Fall,ADVANCED STANDING,Resident,Female,Underrepresented Minority,African American,African American/Black,N,3.0,Y,Y,Y,Y,Y,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,4052,NaN,26000.0,Clg of Environmental Design,Clg of Environmental Design,Landscape Arch & Env Plan,English+Other,...,NaN,NaN,NaN,NaN,Clg of Env Design,Clg of Env Design,Landscape Arch & Env Plan,Landscape Architecture,Clg of Natural Resources,Clg of Natural Resources,"Env Sci, Policy, & Mgmt",Conserv & Resource Stds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014.0,Spring,2013-14,3.871,Clg of Environmental Design,Clg of Environmental Design,Landscape Arch & Env Plan,Landscape Architecture,Clg of Natural Resources,Clg of Natural Resources,"Env Sci, Policy, & Mgmt",Conserv & Resource Stds,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,161295,1,2015,Fall,ADVANCED STANDING,Resident,Male,White/Other,White,White,N,NaN,Y,N,N,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,7711,NaN,90000.0,Clg of Environmental Design,Clg of Environmental Design,City & Regional Planning,English,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,183848,1,2010,Spring,ADVANCED STANDING,Resident,Male,White/Other,White,White,N,2.0,Y,Y,Y,Y,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,U,NaN,4061,NaN,32000.0,Clg of Letters & Science,L&S Arts & Humanities Division,Art Practice,-,...,NaN,NaN,NaN,NaN,Clg of Letters & Science,L&S-Arts & Humanities Div,Art Practice,Art Practice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,Fall,2011-12,3.977,Clg of Letters & Science,L&S Arts & Humanities Division,Art Practice,Art,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,232294,1,2009,Fall,ADVANCED STANDING,Resident,Female,White/Other,Other/Decline to State,Decline to State,N,2.5,Y,Y,Y,Y,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,4052,NaN,39003.0,Clg of Letters & Science,L&S Arts & Humanities Division,Slavic Languages & Literatures,-,...,NaN,NaN,NaN,NaN,Clg of Letters & Science,L&S-Arts & Humanities Div,Slavic Languages & Lit,Slavic Lang & Lit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011.0,Fall,2011-12,3.939,Clg of Letters & Science,L&S Arts & Humanities Division,Slavic Languages & Literatures,Slavic Lang & Lit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,225526,1,2010,Fall,ADVANCED STANDING,Resident,Female,White/Other,White,White,N,2.0,Y,Y,Y,Y,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN,7711,NaN,NaN,Clg of Environmental Design,Clg of Environmental Design,Architecture,English,...,NaN,NaN,NaN,NaN,Clg of Env Design,Clg of Env Design,Architecture,Architecture,NaN,NaN,NaN,NaN,Na

In [9]:
# Be aware, this cell is very resource intensive
course_data.sort()
student_courses = pd.DataFrame()
for f in course_data:
    student_courses = student_courses.append(pd.read_table(f))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [10]:
course_data

['/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.course.data.2007.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.course.data.2008.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.course.data.2009.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.course.data.2010.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.course.data.2011.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.course.data.2012.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.course.data.2013.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.course.data.2014.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.course.data.2015.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.course.data.2016.txt',
 '/content/drive/MyDrive/Colab Notebooks/data/uc.berkeley.student.course.data.2017.txt',
 '/content/drive/MyDr

In [3]:
# minor data cleaning
student_courses = student_courses[~student_courses['ppsk'].isnull()]
student_courses = student_courses[student_courses['ppsk'] != -1]
for i in student_courses.columns:
    if student_courses[i].isnull().all():
        student_courses.drop(i,axis=1,inplace=True)

NameError: name 'student_courses' is not defined

In [12]:
# every student who holds a maker pass
maker_passes = pd.read_table(passes[0])
maker_passes = maker_passes[maker_passes['ppsk'] != 'MISSING']
maker_passes = maker_passes['ppsk'].unique()
maker_passes

array(['724096', '687168', '781707', ..., '4365425', '4365890', '4478634'],
      dtype=object)

In [13]:
student_courses

,year.name,ppsk,undergrad.status,course.department,course.subject,course.number,offering.type,section.number,ccn,room.share,course.title,instructor.name,grade.subtype,grade.type,grade.name,grade.points,student.credit.hours
93,2007 Summer,31341.0,Graduate,Near Eastern Studies,Persian,20,Primary,1,70310,70310.0,Inten Inter Persian,Jaleh Pirnazar,A,Letter Grade,A+,4.0,10.0
94,2007 Summer,33827.0,Graduate,German,German,10,Primary,1,48655,48655.0,Elem German Wrkshp,Meredith Kolar; Nikolaus Euba,A,Letter Grade,A+,4.0,10.0
95,2007 Summer,169335.0,Graduate,Near Eastern Studies,Near Eastern Studies,290A,Independent Study,3,66140,66140.0,Special Studies,Hatem Bazian,A,Letter Grade,A+,4.0,4.0
96,2007 Summer,194536.0,Graduate,Ethnic Studies,Ethnic Studies,135AC,Primary,2,45245,45245.0,U.S. Immigration,Catherine Choy,A,Letter Grade,A+,4.0,4.0
97,2007 Summer,198873.0,Undergraduate,Near Eastern Studies,Hebrew,N106,Primary,1,50010,50010.0,Elem Bibl Hebrew,Chava Boyarin,A,Letter Grade,A+,4.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23102,2019 Summer,4271436.0,Undergraduate,Nutritional Sciences & Tox,Nutritional Science & Tox,10,Primary,2,14561,14561.0,Intro Human Nutr,NaN,B,Letter Grade,B-,2.7,3.0
23103,2019 Summer,4271494.0,Undergraduate,Psychology,Psychology,166AC,Primary,1,14105,14105.0,Cultural Psychology,NaN,A,Letter Grade,A+,4.0,3.0
23104,2019 Summer,4271527.0,Undergraduate,Theater Dance & Perf Stds,Theater Dance & Perf Stds,10,Primary,1,14043,14043.0,Fund. Of Acting I,NaN,A,Letter Grade,A,4.0,3.0
23105,2019 Summer,4271531.0,Undergraduate,Interdisc Social Science Pgms,Global Studies,154M,Primary,3,15353,15353.0,Topics Middle East,NaN,A,Letter Grade,A,4.0,4.0


In [14]:
# all students who took design innovation course
all_jacobs = student_courses[student_courses["course.department"] == "Design Innovation"]['ppsk'].unique()

In [15]:
len(all_jacobs)

647

In [16]:
len(maker_passes)

8449

In [17]:
%store student_courses

Stored 'student_courses' (DataFrame)


In [18]:
student_courses[student_courses["course.subject"] == "Design Innovation"]['year.name'].unique()

array(['2015 Fall', '2015 Spring', '2016 Spring', '2016 Fall',
       '2017 Spring', '2017 Summer', '2017 Fall', '2018 Fall',
       '2018 Spring', '2018 Summer', '2019 Fall', '2019 Spring',
       '2019 Summer'], dtype=object)

In [19]:
full_info = student_courses.merge(pathways, left_on= ['ppsk'])
full_info

TypeError: Can only merge Series or DataFrame objects, a <class 'list'> was passed

In [1]:
# data exploration for student courses
sorted(student_courses['course.subject'].unique())

NameError: name 'student_courses' is not defined

In [2]:
student_courses[student_courses[['grade.subtype', 'grade.type', 'grade.name']]['grade.type'] == 'Non-Letter Grade']['grade.name'].unique()

NameError: name 'student_courses' is not defined